In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import datetime
import tqdm
import random
import requests
import numpy as np
import pandas as pd
import requests

In [2]:

class Content:
    def __init__(self, url, title, price, square, district, datePost, linkDescription, detailDesciption):
        self.url = url
        self.title = title
        self.price = price
        self.square = square
        self.district = district
        self.datePost= datePost 
        self.linkDescription = linkDescription
        self.detailDesciption = detailDesciption

class Website:
    def __init__(self, name, url, titleTag, priceTag, squareTag, districtTag, datePost, linkDescriptionTag, detailDesciptionTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.priceTag = priceTag
        self.squareTag = squareTag
        self.districtTag = districtTag
        self.datePost = datePost
        self.linkDescriptionTag = linkDescriptionTag
        self.detailDesciptionTag = detailDesciptionTag 
class Crawler:
    # Requests to Websites to accept crawl data
    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text, 'html.parser')
    # select tag for Website
    def link_description(self, url, pageDescription, selector):
        try:
            req = requests.get(url+"{}".format(pageDescription))

        except requests.exceptions.RequestException:
            return None
        bs = BeautifulSoup(req.text, 'html.parser')
        return ' '.join([i.get_text() for i in bs.select(selector)])
    def safeGet(self, pageObj, selector):
        selectedElems = pageObj.select(selector)
        if selectedElems is not None and len(selectedElems) > 0:
            data = []
            for elem in selectedElems:             
                if elem.name == 'div' or elem.name == 'p' or elem.name == 'ul' or elem.name == 'li' or  elem.name == 'h3' or elem.name == 'h2' or elem.name == 'h4' or elem.name == 'time' or elem.name =='span':
                    data.append(elem.get_text())
                elif elem.name == 'a':
                    # pageDescription
                    data.append(elem.get('href'))
                if elem.name == 'img' :
                    data.append(elem.get('data-src'))
            return data
        return ''
    # Result data 
    def parse(self, site, url):
        bs = self.getPage(url)
        if bs is not None:
            title = self.safeGet(bs, site.titleTag)
            price = self.safeGet(bs, site.priceTag)
            square = self.safeGet(bs, site.squareTag)
            district = self.safeGet(bs, site.districtTag)
            datePost = self.safeGet(bs, site.datePost)
            linkDescription = self.safeGet(bs, site.linkDescriptionTag)

            detailDesciption = [self.link_description(site.url, link.replace(site.url,''), site.detailDesciptionTag) for link in linkDescription]


        if title != '' and price != '' and square != '' and district != '' and datePost != '' and linkDescription != '' and detailDesciption != '':
            content = Content(url, title, price, square, district, datePost, linkDescription, detailDesciption)
           
            return content
        else:
            print("please recheck elements from {}".format(url))
            return ''
  

In [3]:
crawler = Crawler()

siteData = [
    ['Meey Land', # name 
     'https://meeyland.com', # url
     'div.card-title h3', # titleTag
     'div.article-price', # priceTag
     'div.article-info-detail-item.no-wrap.items-center.icon-area',# squareTag
     'div.article-info-location', #districtTag
     'div.no-wrap.items-center.chip-content-footer.icon-date', # datePost
     'div.card-head-img a[href]', # linkDescriptionTag
     'div.des-detail.ml-text-14 div' # detailDesciptionTag
    ],
    [
        'Mogi',# name 
        'https://mogi.vn', # url
        'ul.props h2.prop-title', # titleTag
        'ul.props div.price', #price Tag
        'ul.props ul.prop-attr li:first-child', # square 
        'ul.props div.prop-addr', # district Tag
        'ul.props div.prop-created', # datePost,
        'ul.props div.prop-info a.link-overlay[href]', #linkDescription,
        'div.info-content-body' # detailDescription         
    ],
    [
        'phongtro213',# name 
        'https://phongtro123.com/', # url
        'ul.post-listing.clearfix h4.post-title', # titleTag
        'ul.post-listing.clearfix span.post-price', #price Tag
        'ul.post-listing.clearfix span.post-acreage', # square 
        'ul.post-listing.clearfix span.post-location', # district Tag
        'ul.post-listing.clearfix time.post-time', # datePost,
        'ul.post-listing.clearfix li.post-item.clearfix.tin-vip a[href]', #linkDescription,
        'div.section-content' # detailDescription         
    ]
    
    
    
]


In [4]:
websites = []
for row in siteData:
    websites.append(Website(row[0], row[1], row[2], row[3],row[4],row[5], row[6], row[7], row[8]))

In [39]:
def multipage(websites, urlPage, tag, numberPage):
    crawler = Crawler()
    data = []
    for num in range(1,numberPage):
        if num == 1:
            
            data.append(crawler.parse(websites, urlPage))
        else:
            page = crawler.parse(websites, urlPage+tag.format(str(num)))
            if page != '':
                data.append(page)
                print(data)
            else:
                break;
    return data     
mogiData = []
district = [7]
for i in district:
    print(i)
    urlPage = "https://phongtro123.com/tinh-thanh/ho-chi-minh/quan-{}?gia=3".format(i)
    tag = '&page={}'
    numberPage = 10
    mogiData.append(multipage(websites[2], urlPage, tag,numberPage))        

# titleTag = [] 
# priceTag = []
# squareTag = []
# districtTag = []
# datePostTag = []
# linkDescriptionTag = []
# detailDescriptionTag = []
# imgTag = []
# for page in mogiData:
#     for i in page:
#         titleTag.append(i.title)
#         priceTag.append(i.price)
#         squareTag.append(i.square)
#         districtTag.append(i.district)
#         datePostTag.append(i.datePost)
#         linkDescriptionTag.append(i.linkDescription)
# #         imgTag.append(i.image)
#         detailDescriptionTag.append(i.detailDesciption)
    

7
please recheck elements from https://phongtro123.com/tinh-thanh/ho-chi-minh/quan-7?gia=3
please recheck elements from https://phongtro123.com/tinh-thanh/ho-chi-minh/quan-7?gia=3&page=2


In [6]:
from tqdm import tqdm
  
  
crawler = Crawler()
websites[2]
data = []
for i in tqdm(range(int(20))):
    print('https://mogi.vn/thue-phong-tro-nha-tro?cp={}'.format(i))
    data.append(crawler.parse(websites[1], 'https://mogi.vn/thue-phong-tro-nha-tro?cp={}'.format(i)))

  0%|          | 0/20 [00:00<?, ?it/s]

https://mogi.vn/thue-phong-tro-nha-tro?cp=0


  5%|▌         | 1/20 [00:05<01:52,  5.90s/it]

https://mogi.vn/thue-phong-tro-nha-tro?cp=1


 10%|█         | 2/20 [00:09<01:26,  4.81s/it]

...

 90%|█████████ | 18/20 [01:15<00:08,  4.05s/it]

https://mogi.vn/thue-phong-tro-nha-tro?cp=18


 95%|█████████▌| 19/20 [01:19<00:03,  3.97s/it]

https://mogi.vn/thue-phong-tro-nha-tro?cp=19


100%|██████████| 20/20 [01:23<00:00,  4.17s/it]


In [13]:
for page in data:
    print(page.title)
    

['🏢Khai trương Phòng Phú Nhuận Full NT có bồn tắm siêu đẹp,ban công🌟', '🏢Khai trương Siêu phẩm Phòng Tân Bình có bồn tắm siêu đẹp,ban công🌟', '🏢Khai trương Siêu phẩm Phòng Quận 1 có bồn tắm siêu đẹp, ban công🌟', '🏢Khai trương Siêu phẩm Phòng Quận 11 có bồn tắm siêu đẹp, ban công🌟', '🏢Khai trương Siêu phẩm Phòng Quận 5 có bồn tắm siêu đẹp, ban công🌟', '🏢Khai trương Siêu phẩm Phòng Quận 3 có bồn tắm siêu đẹp, ban công🌟', '🏢Khai trương Siêu phẩm Phòng  Quận 10 có bồn tắm siêu đẹp, ban công🌟', 'Phòng cao cấp 3 sao, máy giặt riêng, gần cầu Công Lý Q.3, KM lớn T.6', 'Phòng cao cấp chuẩn căn hộ, Nguyễn Văn Trỗi, 5 phút đi sân bay', 'Phòng cho thuê có máy lạnh, hẻm ô tô, giờ tự do, Hồ Văn Huê', '⁉️Phòng trọ cao cấp giá sinh viên _ nội thất đầy đủ ⁉️', 'Phòng trọ giá rẻ quận 10 full nội thất, gần tô hiến thành, cv thị riên', '?Khai Trương phòng trọ_Nguyễn Văn Khối_Trường cao đẳng Miền Nam_GIÁTỐT', 'Cho thuê căn hộ cao cấp Phạm Văn Xão_Vườn Lài_Quận Tân Phú_GIÁ TỐT', 'Phòng Trọ có ban công Âu Cơ

In [17]:

def dataframe(data):
    titleTag = [] 
    priceTag = []
    squareTag = []
    districtTag = []
    datePostTag = []
    linkDescriptionTag = []
    detailDescriptionTag = []
    imgTag = []
    for i in data:
            titleTag.append(i.title)
            priceTag.append(i.price)
            squareTag.append(i.square)
            districtTag.append(i.district)
            datePostTag.append(i.datePost)
            linkDescriptionTag.append(i.linkDescription)
            detailDescriptionTag.append(i.detailDesciption)
            
    data = {"title":sum(titleTag, []), 
            "price": sum(priceTag, []), 
            "square": sum(squareTag, []),
            "district": sum(districtTag, []), 
            "datePost": sum(datePostTag, []),
            "linkDescription": sum(linkDescriptionTag, []),
            "detailDescription": sum(detailDescriptionTag, [])}
    print(len(data['title']), len(data['price']), len(data['square']), len(data['district']), len(data['datePost']))
    print(len(data['linkDescription']), len(data['detailDescription']))
    return data

data = dataframe(data)
# data = pd.DataFrame(data = data)

AttributeError: 'str' object has no attribute 'price'

In [23]:
pd.DataFrame(data)

,title,price,square,district,datePost,linkDescription,detailDescription
0,🏢Khai trương Phòng Phú Nhuận Full NT có bồn tắ...,6 triệu,35 m2,,Hôm nay,https://mogi.vn/quan-phu-nhuan/thue-phong-tro-...,🏢🏢🏢Khai trương Siêu phẩm Căn hộ đang hot Quận ...
1,🏢Khai trương Siêu phẩm Phòng Tân Bình có bồn t...,6 triệu,35 m2,,Hôm nay,https://mogi.vn/quan-tan-binh/thue-phong-tro-k...,🏢🏢🏢Khai trương Siêu phẩm Căn hộ đang hot Quận ...
2,🏢Khai trương Siêu phẩm Phòng Quận 1 có bồn tắm...,6 triệu,35 m2,,Hôm nay,https://mogi.vn/quan-1/thue-phong-tro-khu-nha-...,🏢🏢🏢Khai trương Siêu phẩm Căn hộ đang hot Quận ...
3,🏢Khai trương Siêu phẩm Phòng Quận 11 có bồn tắ...,6 triệu,35 m2,,Hôm nay,https://mogi.vn/quan-11/thue-phong-tro-khu-nha...,🏢🏢🏢Khai trương Siêu phẩm Căn hộ đang hot Quận ...
4,🏢Khai trương Siêu phẩm Phòng Quận 5 có bồn tắm...,6 triệu,35 m2,,Hôm nay,https://mogi.vn/quan-5/thue-phong-tro-khu-nha-...,🏢🏢🏢Khai trương Siêu phẩm Căn hộ đang hot Quận ...
...,...,...,...,...,...,...,...
295,KTX có máy lạnh cho thuê - Bình Thạnh 45m²,700.000 đ,56 m2,,Hôm nay,https://mogi.vn/quan-binh-thanh/thue-phong-tro...,ký túc xá cao cấp quận bình thạnh giá rẻ thuộc...
296,KTX cao cấp giá 700k/ giường tại 207 ngô tất t...,700.000 đ,45 m2,,Hôm nay,https://mogi.vn/quan-binh-thanh/thue-phong-tro...,phòng kí túc xá dành cho người mới đi làm giá ...
297,Phòng NEW100% FULL nội thất Huỳnh Văn Nghệ Tân...,3 triệu 600 nghìn,30 m2,,Hôm nay,https://mogi.vn/quan-tan-binh/thue-phong-tro-k...,"Cho thuê phòng mới xây 100% có cửa sổ, ban côn..."
298,"Căn Hộ Cao Cấp Máy Giặc Riêng - TÔ HIẾN THÀNH,...",6 triệu 500 nghìn,30 m2,,Hôm nay,https://mogi.vn/quan-10/thue-phong-tro-khu-nha...,"Địa Chỉ: 451/24 Tô Hiến Thành, Phường 14, Quận..."


In [1]:

# data = dataframe(mogiData,"mogi.csv")
# pd.set_option('display.max_colwidth',None)
# data = pd.DataFrame(data =data)

# district =['quan-1','quan-3','quan-5','quan-7']
# for i in range(len(data['linkDescription'])):
#     for j in district:
#         if j in data['linkDescription'].iloc[i]:
#             data['district'].iloc[i] = j
    
# data['datePost'] = data['datePost'].str.replace('Hôm nay','19/03/2021')
# data['datePost'] = data['datePost'].str.replace('Hôm qua','18/03/2021')
# data.to_csv("Mogi.csv")
# data.head()


In [22]:
#         'mogi',# name 
#         'https://mogi.vn', # url
#         'h2.prop-title', # titleTag
#         'div.price', #price Tag
#         'ul.prop-attr li:first-child', # square 
#         'div.prop-addr', # district Tag
#         'div.prop-created' # datePost,
#         'a.link-overlay', #linkDescription,
#         'div.info-content-body' # detailDescription       
#     [
#         'phongtro213',# name 
#         'https://phongtro123.com/', # url
#         'h4.post-title', # titleTag
#         'span.post-price', #price Tag
#         'span.post-acreage', # square 
#         'span.post-location', # district Tag
#         'time.post-time', # datePost,
#         'li.post-item.clearfix.tin-vip a[href]', #linkDescription,
#         'div.section-content' # detailDescription         
#     ]
    
    
url = "https://phongtro123.com/tinh-thanh/ho-chi-minh/cho-thue-phong-cao-cap-trung-tam-quan-7.html"

try:
    req = requests.get(url)
    print("it's okay")
    bs = BeautifulSoup(req.text, 'html.parser')
except requests.exceptions.RequestException:
    print("huhu")
for i in bs.select("div.section-content"):
    print(i.get_text())


it's okay
Cho thuê cao cấp tại : 176/1A Nguyễn Thị Thập, P.Bình thuận, Q7( Xe hơi vào tận cửa)Phòng mới xây, sạch sẽ, thoáng mát, giờ tự do... có gác suốtCách chợ Tân Mỹ, Phú Mỹ Hưng 2 phút, siêu thị Lotte, trường ĐH Tôn Đức Thắng, ĐH Marketing 5 phút.Cho thuê phòng trống, có máy lạnh hoặc có thêm nội thất tùy theo nhu cầu khách hàng.Gía thuê phòng từ 2.2tr không gác và 3tr có gác.
Mã tin:#286074Khu vựcCho thuê Phòng trọ Hồ Chí MinhLoại tin rao:Phòng trọ, nhà trọĐối tượng thuê:Tất cảGói tin:Tin thường
Liên hệ: thinhminh1212Điện thoại: 0902408086

Mọi thông tin trên website chỉ mang tính chất tham khảo. Nếu bạn có phản hồi với tin đăng này (báo xấu, tin đã cho thuê, không liên lạc được,...), vui lòng thông báo để PhòngTrọ123 có thể xử lý. Gửi phản hồi
Liên hệ với chúng tôi nếu bạn cần hỗ trợ:Hỗ trợ thanh toánĐiện thoại: 0917686101Zalo: 0917686101Hỗ trợ đăng tinĐiện thoại: 0902657123Zalo: 0902657123Hotline 24/7Điện thoại: 0917686101Zalo: 0917686101Gửi liên hệ


In [318]:
bs

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=0, shrink-to-fit=no" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/html;charset=utf-8" http-equiv="Content-type"/>
<meta content="vi" http-equiv="content-language"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="/assets/css/app-796288e240ecc2293e00.css" rel="stylesheet"/>
<link href="https://s3.prod.now.vn/resources/shopee_font/font.css" rel="stylesheet"/>
<!-- Google Tag Manager -->
<script>
      (function(w, d, s, l, i) {
        w[l] = w[l] || [];
        w[l].push({ 'gtm.start': new Date().getTime(), event: 'gtm.js' });
        var f = d.getElementsByTagName(s)[0],
          j = d.createElement(s),
          dl = l != 'dataLayer' ? '&l=' + l : '';
        j.async = true;
        j.src = 'https://www.googletagmanager.com/gtm.js?id=' + i + dl;
        f.parentNode.